In [145]:
import numpy as np
import cv2
import matplotlib.pyplot as plt
import matplotlib.image as mpimg

In [146]:
def output_corners():
    for i in range(1,21):
        inpath = "./camera_cal/calibration%d.jpg" % i
        outpath = "./output_images/corners%d.jpg" % i
        
        nx = 9
        ny = 5 if i in [1, 4, 5] else 6
        img = cv2.imread(inpath)
        gray = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
        ret, corners = cv2.findChessboardCorners(gray, (nx, ny), None)
        if ret:
            cv2.drawChessboardCorners(img, (nx, ny), corners, ret)
            cv2.imwrite(outpath, img)


In [147]:
def catalog_images():
    imgs = []
    for i in range(1, 2):
        inf = {}
        inf["path"] = "./camera_cal/calibration%d.jpg" % i
        inf["undist_path"] = "./output_images/calibration_undist%d.jpg" % i
        inf["nx"] = 9
        inf["ny"] = 5 if i == 1 else 6
        imgs.append(inf)
    return imgs

def calculate_undist():
    # make objpoints
    # find corners
    # calculate distortion
    # undistort

    imgs = catalog_images()
    objpoints = []
    imgpoints = []

    for inf in imgs:
        nx = inf["nx"]
        ny = inf["ny"]
        imgpath = inf["path"]
    
        objp = np.zeros((nx*ny,3), np.float32)
        objp[:,:2] = np.mgrid[0:nx, 0:ny].T.reshape(-1,2)
        
        img = cv2.imread(inf["path"])
        gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
        
        img_size = gray.shape[::-1]
        if img_size != (1280, 720):
            continue

        ret, imgp = cv2.findChessboardCorners(gray, (nx, ny), None)

        if not ret:
            continue
            
        objpoints.append(objp)
        imgpoints.append(imgp)
        
    ret, mtx, dist, rvecs, tvecs = cv2.calibrateCamera([objp], [imgp], gray.shape[::-1], None, None)
    
    for inf in imgs:
        nx = inf["nx"]
        ny = inf["ny"]
        imgpath = inf["path"]
        img = cv2.imread(inf["path"])
        gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
        img_size = gray.shape[::-1]
        if img_size != (1280, 720):
            continue
        undist = cv2.undistort(img, mtx, dist, None, mtx)
        cv2.imwrite(inf["undist_path"], undist)
        
    return ret, mtx, dist, rvecs, tvecs


def test_images():
    return ["./test_images/straight_lines%d.jpg" % i for i in [1, 2]] + \
           ["./test_images/test%d.jpg" % i for i in range(1,7)]
    
def trapezoid():
    res = np.float32([(457, 1400), (1190, 900), (1367, 900), (2150, 1400)])/2
    
    # made from undist_straight_lines2.jpg
    # res = np.float32([(529, 1370), (1218, 876), (1342, 876), (2120, 1370)])/2
    return res

def trapezoid_dst():
    
    res = np.float32([(457, 1400), (457, 100), (2150, 100), (2150, 1400)])/2
    
    # res = np.float32([(529, 1370), (529, 876), (2120, 876), (2120, 1370)])/2
    return res

    
def show_undist():
    ret, mtx, dist, rvecs, tvecs = calculate_undist()
    ti = test_images()
    for path in ti:
        img = cv2.imread(path)
        try:
            undist = cv2.undistort(img, mtx, dist, None, mtx)
        except:
            print(path)
            print(img)
            
        outpath = path.replace("test_images/", "output_images/undist_")
        cv2.imwrite(outpath, undist)
        
def make_perspective_transform():
    return cv2.getPerspectiveTransform(trapezoid(), trapezoid_dst())

def make_inv_perspective_transform():
    return cv2.getPerspectiveTransform(trapezoid_dst(), trapezoid())


def de_perspect(img, M):
    img_size = (img.shape[1], img.shape[0])
    return cv2.warpPerspective(img, M, img_size, flags=cv2.INTER_LINEAR)
    


In [148]:
show_undist()

In [149]:
output_corners()    
    

In [150]:
trapezoid()

array([[  228.5,   700. ],
       [  595. ,   450. ],
       [  683.5,   450. ],
       [ 1075. ,   700. ]], dtype=float32)

In [151]:
path = "./output_images/undist_straight_lines1.jpg"
img = cv2.imread(path)
%matplotlib inline

M = make_perspective_transform()
M_inv = make_inv_perspective_transform()
warped = de_perspect(img, M)

cv2.imwrite(path.replace("undist", "de_perspect"), warped)
    

True

In [152]:
def magic_channel(img):
    hls = cv2.cvtColor(img, cv2.COLOR_RGB2HLS).astype(np.float)
    
    ch1 = hls[:,:,2]
    ch2 = img[:,:,2]
    
    magic = ch1 ####  (ch1 + ch2) * ch1
    magic = np.uint8(255 * magic/np.max(magic))
    return magic    

def edgify(magic):
    
    s = cv2.Sobel(magic, cv2.CV_64F, 1, 0)
    s = np.absolute(s)
    s = np.uint8(255 * s/np.max(s))
    
    res = np.zeros_like(s)
    res[(s > 30) & (s < 200)] = 255
    
    return res



In [153]:

def find_and_fit_poly(binary_warped, poly_degree=2):
    histogram = np.sum(binary_warped[binary_warped.shape[0]//2:,:], axis=0)

    # Find the peak of the left and right halves of the histogram
    # These will be the starting point for the left and right lines
    midpoint = np.int(histogram.shape[0]/2)
    leftx_base = np.argmax(histogram[:midpoint])
    rightx_base = np.argmax(histogram[midpoint:]) + midpoint

    # Choose the number of sliding windows
    nwindows = 9
    # Set height of windows
    window_height = np.int(binary_warped.shape[0]/nwindows)
    # Identify the x and y positions of all nonzero pixels in the image
    nonzero = binary_warped.nonzero()
    nonzeroy = np.array(nonzero[0])
    nonzerox = np.array(nonzero[1])
    # Current positions to be updated for each window
    leftx_current = leftx_base
    rightx_current = rightx_base
    # Set the width of the windows +/- margin
    margin = 100
    # Set minimum number of pixels found to recenter window
    minpix = 50
    # Create empty lists to receive left and right lane pixel indices
    left_lane_inds = []
    right_lane_inds = []

    # Step through the windows one by one
    for window in range(nwindows):
        # Identify window boundaries in x and y (and right and left)
        win_y_low = binary_warped.shape[0] - (window+1)*window_height
        win_y_high = binary_warped.shape[0] - window*window_height
        win_xleft_low = leftx_current - margin
        win_xleft_high = leftx_current + margin
        win_xright_low = rightx_current - margin
        win_xright_high = rightx_current + margin
        # Identify the nonzero pixels in x and y within the window
        good_left_inds = ((nonzeroy >= win_y_low) & (nonzeroy < win_y_high) & (nonzerox >= win_xleft_low) & (nonzerox < win_xleft_high)).nonzero()[0]
        good_right_inds = ((nonzeroy >= win_y_low) & (nonzeroy < win_y_high) & (nonzerox >= win_xright_low) & (nonzerox < win_xright_high)).nonzero()[0]
        # Append these indices to the lists
        left_lane_inds.append(good_left_inds)
        right_lane_inds.append(good_right_inds)
        # If you found > minpix pixels, recenter next window on their mean position
        if len(good_left_inds) > minpix:
            leftx_current = np.int(np.mean(nonzerox[good_left_inds]))
        if len(good_right_inds) > minpix:        
            rightx_current = np.int(np.mean(nonzerox[good_right_inds]))

    # Concatenate the arrays of indices
    left_lane_inds = np.concatenate(left_lane_inds)
    right_lane_inds = np.concatenate(right_lane_inds)

    # Extract left and right line pixel positions
    leftx = nonzerox[left_lane_inds]
    lefty = nonzeroy[left_lane_inds] 
    rightx = nonzerox[right_lane_inds]
    righty = nonzeroy[right_lane_inds] 

    # Fit a second order polynomial to each
    left_fit = np.polyfit(lefty, leftx, poly_degree)
    right_fit = np.polyfit(righty, rightx, poly_degree)
    return left_fit, right_fit, nonzerox, nonzeroy, left_lane_inds, right_lane_inds

In [154]:
def draw_lane(binary_warped, left_fit, right_fit, nonzerox, nonzeroy, left_lane_inds, right_lane_inds):
    
    # Generate x and y values for plotting
    out_img = np.dstack((binary_warped, binary_warped, binary_warped))*255
    ploty = np.linspace(0, binary_warped.shape[0]-1, binary_warped.shape[0] )
    left_fitx = left_fit[0]*ploty**2 + left_fit[1]*ploty + left_fit[2]
    right_fitx = right_fit[0]*ploty**2 + right_fit[1]*ploty + right_fit[2]

    out_img[nonzeroy[left_lane_inds], nonzerox[left_lane_inds]] = [255, 0, 0]
    out_img[nonzeroy[right_lane_inds], nonzerox[right_lane_inds]] = [0, 0, 255]

    # Create an image to draw on and an image to show the selection window
    window_img = np.zeros_like(out_img)
    # Color in left and right line pixels
    out_img[nonzeroy[left_lane_inds], nonzerox[left_lane_inds]] = [255, 0, 0]
    out_img[nonzeroy[right_lane_inds], nonzerox[right_lane_inds]] = [0, 0, 255]

    # Generate a polygon to illustrate the search window area
    # And recast the x and y points into usable format for cv2.fillPoly()
    left_line_window1 = np.array([np.transpose(np.vstack([left_fitx-margin, ploty]))])
    left_line_window2 = np.array([np.flipud(np.transpose(np.vstack([left_fitx+margin, ploty])))])
    left_line_pts = np.hstack((left_line_window1, left_line_window2))
    right_line_window1 = np.array([np.transpose(np.vstack([right_fitx-margin, ploty]))])
    right_line_window2 = np.array([np.flipud(np.transpose(np.vstack([right_fitx+margin, ploty])))])
    right_line_pts = np.hstack((right_line_window1, right_line_window2))

    # Draw the lane onto the warped blank image
    cv2.fillPoly(window_img, np.int_([left_line_pts]), (0,255, 0))
    cv2.fillPoly(window_img, np.int_([right_line_pts]), (0,255, 0))
    result = cv2.addWeighted(out_img, 1, window_img, 0.3, 0)

    return window_img

In [155]:
def curvature_and_shift_calc(binary_warped, left_fit, right_fit):
    # Define conversions in x and y from pixels space to meters
    ym_per_pix = 30/720 # meters per pixel in y dimension
    xm_per_pix = 3.7/700 # meters per pixel in x dimension
    
    ploty = np.linspace(0, binary_warped.shape[0]-1, binary_warped.shape[0] )
    midpoint = binary_warped.shape[1] / 2
    y_eval = np.max(ploty)
    
    left_fitx = left_fit[0]*ploty**2 + left_fit[1]*ploty + left_fit[2]
    right_fitx = right_fit[0]*ploty**2 + right_fit[1]*ploty + right_fit[2]
    
    lane_midpoint = (left_fitx[-1] + right_fitx[-1]) / 2
    shift = (midpoint - lane_midpoint) * xm_per_pix

    # Fit new polynomials to x,y in world space
    left_fit_cr = np.polyfit(ploty*ym_per_pix, left_fitx*xm_per_pix, 2)
    right_fit_cr = np.polyfit(ploty*ym_per_pix, right_fitx*xm_per_pix, 2)
    
    
    # Calculate the new radii of curvature
    left_curverad = ((1 + (2*left_fit_cr[0]*y_eval*ym_per_pix + left_fit_cr[1])**2)**1.5) / np.absolute(2*left_fit_cr[0])
    right_curverad = ((1 + (2*right_fit_cr[0]*y_eval*ym_per_pix + right_fit_cr[1])**2)**1.5) / np.absolute(2*right_fit_cr[0])
    # Now our radius of curvature is in meters
    return left_curverad, right_curverad, shift

tests = ["./output_images/undist_test%d.jpg" % i for i in range(1, 7)]

def save_img(img, name_part, to_replace="undist"):
    cv2.imwrite(path.replace(to_replace, name_part), img)

for path in tests:
    img = cv2.imread(path)
    
    magic = magic_channel(img)
    save_img(magic, "magic")
    
    e = edgify(magic)
    save_img(e, "edgy")
    
    warped = de_perspect(e, M)
    save_img(warped, "edgy_unwarp")
    
    left_fit, right_fit, nonzerox, nonzeroy, left_lane_inds, right_lane_inds = find_and_fit_poly(warped, 2)
    
    print(curvature_and_shift_calc(warped, left_fit, right_fit))
    
    lane = draw_lane(warped, left_fit, right_fit, nonzerox, nonzeroy, left_lane_inds, right_lane_inds)
    save_img(lane, "lane")
    
      
    re_perspect = de_perspect(lane, M_inv)

    blended = cv2.addWeighted(img, 1, re_perspect, 0.3, 0)
    save_img(blended, "blended")



(521.42864478902038, 812.16381540329212, -0.29408549843985471)
(335.85167516710152, 132.69286133743648, -0.70116486851578474)
(2727.8277275004234, 159.88384507665253, -0.22962557842296191)
(229.04565863737957, 1241.8429796448418, -0.4086735166741014)
(120.21480812066542, 324.23637328910428, -0.24585420626977542)
(344.35818385292481, 2320.158033190206, -0.36722028486130509)


In [156]:
import imageio
imageio.plugins.ffmpeg.download()
from moviepy.editor import VideoFileClip
from IPython.display import HTML

In [157]:
ret, mtx, dist, rvecs, tvecs = calculate_undist()
M = make_perspective_transform()
M_inv = make_inv_perspective_transform()

def full_pipeline(img):
    img = cv2.undistort(img, mtx, dist, None, mtx)
    blended = img
    
    try:
        magic = magic_channel(img)
        edges = edgify(magic)
        straight = de_perspect(edges, M)
        left_fit, right_fit, nonzerox, nonzeroy, left_lane_inds, right_lane_inds = find_and_fit_poly(straight, 2)
        lane = draw_lane(warped, left_fit, right_fit, nonzerox, nonzeroy, left_lane_inds, right_lane_inds)

        perspectivized = de_perspect(lane, M_inv)
        blended = cv2.addWeighted(img, 1, perspectivized, 0.3, 0)
    except:
        pass
    
    return blended


#img = cv2.imread("./test_images/test1.jpg")
#blended = full_pipeline(img)
#cv2.imwrite("./output_images/full1.jpg", blended)
   

vid = VideoFileClip("./project_video.mp4")
lane_vid = vid.fl_image(full_pipeline) 
%time lane_vid.write_videofile("./project_video_out.mp4", audio=False)


[MoviePy] >>>> Building video ./project_video_out.mp4
[MoviePy] Writing video ./project_video_out.mp4



100%|█████████▉| 1260/1261 [02:11<00:00,  6.66it/s]


[MoviePy] Done.
[MoviePy] >>>> Video ready: ./project_video_out.mp4 

CPU times: user 4min 1s, sys: 4.81 s, total: 4min 6s
Wall time: 2min 12s
